<a href="https://colab.research.google.com/github/thegeeklife/ML_Assignments/blob/main/EnsembleAndBlenders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### CNIT 623 003 Machine Learning 
#### Homework 5
#### Subia Ansari

In [ ]:
import sys
import sklearn
import numpy as np
import os

np.random.seed(623)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)

In [ ]:
X, y = mnist['data'], mnist['target']
X.shape

(70000, 784)

In [ ]:
# convert y from string to integer
y = y.astype(np.uint8)

### Splitting into train, test, validaiton, holdout sets.

`[stacking]:  Use the hold-out set H (20,000) from HW4 as the new training set for a blender function of those three classifiers trained in HW4. Choose two different classifier as your choice of the blender,  which takes the input vector from the predictions of layer 1 classifiers on the new 20,000 training set. Evaluate the results of stacking on the validation set V (10,000) and choose the better blender of the two as the final stacking implementation. Evaluate the results of each individual classifier and the stacking classifier over the test set T(10,000).  How does the stacking implementation compare to the soft voting classifier in HW4?`

In [ ]:
#test set 10k instances
X_h_v , X_test, y_h_v, y_test = train_test_split(X,y,test_size=10000, 
                                           random_state = 623)

#validation : 10K
X_train_h, X_val, y_train_h, y_val = train_test_split(X_h_v, y_h_v, test_size = 10000, random_state=623)

#Training and holdout sets : 30k train; 20k holdout
X_train, X_hold, y_train, y_hold = train_test_split(X_train_h, y_train_h, test_size = 20000, random_state=623)

 In homework 4, I created an ensemble comprising of KNN Classifier, Decision Tree Classifier  and Logisitc Regression after removing the weakest learner, Naive Bayes Classifier. The ensemble gave an accuracy of 93.9% (hard voting)

We will use the same three models now for our L1 Classifier -

1. Logistic Regression
2. Decision Tree
3. KNN Classifier

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# scale the data so that we don't get convergence warning
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

#Scaling the data before training
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_hold_scaled = scaler.transform(X_hold)

#Declaring classifiers
log_clf = LogisticRegression(solver='lbfgs', random_state=623)
dtree_clf = DecisionTreeClassifier(random_state=623) #using default gini and max depth
knn_clf = KNeighborsClassifier(weights='distance', n_neighbors=4, n_jobs=-1)


In [ ]:
classifiers = [log_clf, dtree_clf, knn_clf]

for clf in classifiers:
  print("Training the ", clf)
  clf.fit(X_train_scaled, y_train)

Training the  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=623, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Training the  DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=623, splitter='best')
Training the  KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=4, p=2,
                     weights='distance')


In [ ]:
#Scores of the models

scores = [clf.score(X_val_scaled, y_val) for clf in classifiers]

In [ ]:
scores

[0.9051, 0.8517, 0.9421]

Layer 1 predictions on hold out set

In [ ]:
X_hold_pred = np.empty((len(X_hold_scaled), len(classifiers)), dtype=np.float32)


for index, clf in enumerate(classifiers):
  X_hold_pred[:, index] = clf.predict(X_hold_scaled)
  


In [ ]:
hold_scores = [clf.score(X_hold_scaled, y_hold) for clf in classifiers]
hold_scores

[0.90155, 0.8506, 0.941]

### Use Blender

I am choosing Random Forest and XGBoost as my blenders.

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier


rf_blender = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=623)
gb_blender = GradientBoostingClassifier(n_estimators=200, random_state=623)

blenders = [rf_blender, gb_blender]

rf_blender.fit(X_hold_pred, y_hold)
gb_blender.fit(X_hold_pred, y_hold)

b_rf_score = rf_blender.oob_score_ #oob score
gen_rf_score = rf_blender.score(X_hold_pred, y_hold) #score to compare with Gradient Boosting Classifier

print('Random forest scores - oob : ', b_rf_score, ' Score : ', gen_rf_score)
#for b in blenders:
 # b.fit(X_hold_pred, y_hold)
  #b_scores = [b.oob_score_ for b in blenders]

b_gb_score = gb_blender.score(X_hold_pred, y_hold)
print('Gradient Boosting Classifier Score : ', b_gb_score)


Random forest scores - oob :  0.94465  Score :  0.9585
Gradient Boosting Classifier Score :  0.95425


I would hence choose GradientBoostingClassifier because it has higher accuracy. However, I will still produce the output for training Random Forest on the test data as well.

Generating prediction results over 10K test data from the three classifiers in layer 1.

In [ ]:
X_test_predictions = np.empty((len(X_test), len(classifiers)), dtype=np.float32)

for index, clf in enumerate(classifiers):
  X_test_predictions[:, index] = clf.predict(X_test)

y_pred_rf = rf_blender.predict(X_test_predictions)
y_pred_gb = gb_blender.predict(X_test_predictions)

from sklearn.metrics import accuracy_score
rf_acc = accuracy_score(y_test, y_pred_rf)
gb_acc = accuracy_score(y_test, y_pred_gb)

print('1. Rf Accuracy = ', rf_acc, ' | 2. Gb Accuracy = ', gb_acc)

1. Rf Accuracy =  0.7906  | 2. Gb Accuracy =  0.8023
